# 911 Calls Descriptive Analysis

For this capstone project we will be analyzing some 911 call data from [Kaggle](https://www.kaggle.com/mchirico/montcoalert). The data contains the following fields:

* lat : String variable, Latitude
* lng: String variable, Longitude
* desc: String variable, Description of the Emergency Call
* zip: String variable, Zipcode
* title: String variable, Title
* timeStamp: String variable, YYYY-MM-DD HH:MM:SS
* twp: String variable, Township
* addr: String variable, Address
* e: String variable, Dummy variable (always 1)

Just go along with this notebook and try to complete the instructions or answer the questions in bold using your Python and Data Science skills!

## Data and Setup

____
** Import numpy and pandas **

In [ ]:
import numpy as np
import pandas as pd

** Import visualization libraries and set %matplotlib inline. **

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

** Read in the csv file as a dataframe called df **

In [ ]:
df=pd.read_csv("../input/montcoalert/911.csv")

** Check the info() of the df **

In [ ]:
df.info()

** Check the head of df **

In [ ]:
df.head(3)

## Basic Questions

** What are the top 5 zipcodes for 911 calls? **

In [ ]:
df.zip.value_counts().head(5)

** What are the top 5 townships (twp) for 911 calls? **

In [ ]:
df.twp.value_counts().head(5)

** Take a look at the 'title' column, how many unique title codes are there? **

In [ ]:
df.title.nunique()

## Creating new features

** In the titles column there are "Reasons/Departments" specified before the title code. These are EMS, Fire, and Traffic. Use .apply() with a custom lambda expression to create a new column called "Reason" that contains this string value.** 

**For example, if the title column value is EMS: BACK PAINS/INJURY , the Reason column value would be EMS. **

In [ ]:
df["Reason"]=df.title.apply(lambda x:x.split(":")[0])
df.head(3)

** What is the most common Reason for a 911 call based off of this new column? **

In [ ]:
df.Reason.value_counts()

** Now use seaborn to create a countplot of 911 calls by Reason. **

In [ ]:
sns.countplot(df.Reason);

In [ ]:
sns.displot(df.Reason);

___
** Now let us begin to focus on time information. What is the data type of the objects in the timeStamp column? **

In [ ]:
type(df.timeStamp.iloc[0])

** You should have seen that these timestamps are still strings. Use [pd.to_datetime](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html) to convert the column from strings to DateTime objects. **

In [ ]:
df.timeStamp=pd.to_datetime(df.timeStamp)

In [ ]:
type(df.timeStamp.iloc[0])

** You can now grab specific attributes from a Datetime object by calling them. For example:**

    time = df['timeStamp'].iloc[0]
    time.hour

**You can use Jupyter's tab method to explore the various attributes you can call. Now that the timestamp column are actually DateTime objects, use .apply() to create 3 new columns called Hour, Month, and Day of Week. You will create these columns based off of the timeStamp column, reference the solutions if you get stuck on this step.**

In [ ]:
df["Hour"]=df.timeStamp.apply(lambda x: x.hour)
df["Month"]=df.timeStamp.apply(lambda x: x.month)
df["Day_of_Week"]=df.timeStamp.apply(lambda x: x.dayofweek)


** Notice how the Day of Week is an integer 0-6. Use the .map() with this dictionary to map the actual string names to the day of the week: **

    dmap = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}

In [ ]:
dmap = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}
df["Day_of_Week"]=df["Day_of_Week"].map(lambda x:dmap[x])

In [ ]:
df["Day_of_Week"]

** Now use seaborn to create a countplot of the Day of Week column with the hue based off of the Reason column. **

In [ ]:
sns.countplot(x=df["Day_of_Week"], hue=df.Reason);
# To relocate the legend
plt.legend(bbox_to_anchor=(1.05, 0.2), loc=2, borderaxespad=0.);

In [ ]:
sns.countplot(x="Day_of_Week", data=df, hue=df.Reason)
# To relocate the legend
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.);

**Now do the same for Month:**

In [ ]:
sns.countplot(x="Month", data=df, hue=df.Reason)
# To relocate the legend
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.);

**Did you notice something strange about the Plot?**

_____

** You should have noticed it was missing some Months, let's see if we can maybe fill in this information by plotting the information in another way, possibly a simple line plot that fills in the missing months, in order to do this, we'll need to do some work with pandas... **

** Now create a gropuby object called byMonth, where you group the DataFrame by the month column and use the count() method for aggregation. Use the head() method on this returned DataFrame. **

In [ ]:
byMonth=df.groupby("Month").count()
byMonth

** Now create a simple plot off of the dataframe indicating the count of calls per month. **

In [ ]:
byMonth.Day_of_Week.plot();

** Now see if you can use seaborn's lmplot() to create a linear fit on the number of calls per month. Keep in mind you may need to reset the index to a column. **

In [ ]:
sns.lmplot(x='Month',y='twp',data=byMonth.reset_index());
# for using "lmplot" function, you have to reset index. It must have default indexes.

**Create a new column called 'Date' that contains the date from the timeStamp column. You'll need to use apply along with the .date() method. ** 

In [ ]:
df["Date"]=df.timeStamp.apply(lambda x:x.date())
type(df.Date.iloc[0])

In [ ]:
df.head(2)

** Now groupby this Date column with the count() aggregate and create a plot of counts of 911 calls.**

In [ ]:
df.groupby("Date").count()["twp"].plot(figsize=(10,6))
plt.tight_layout;

In [ ]:
# alternative
df.groupby("Date").twp.count().plot(figsize=(10,6))
plt.tight_layout;

** Now recreate this plot but create 3 separate plots with each plot representing a Reason for the 911 call**

In [ ]:
df.Reason.value_counts()

In [ ]:
df[df.Reason=="EMS"].groupby("Date").twp.count().plot(figsize=(8,5));

In [ ]:
df[df.Reason=="Traffic"].groupby("Date").twp.count().plot(figsize=(8,5));

In [ ]:
df[df.Reason=="Fire"].groupby("Date").twp.count().plot(figsize=(8,5));

____
** Now let's move on to creating  heatmaps with seaborn and our data. We'll first need to restructure the dataframe so that the columns become the Hours and the Index becomes the Day of the Week. There are lots of ways to do this, but I would recommend trying to combine groupby with an [unstack](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.unstack.html) method. Reference the solutions if you get stuck on this!**

In [ ]:
df.Reason.value_counts()

In [ ]:
# examine "stack() / unstack() / pivot_table()" methods !!!

dayHour = df.groupby(['Day_of_Week','Hour']).Reason.count().unstack()
# dayHour = df.groupby(['Day_of_Week','Hour'])["Reason"].count().unstack()
# dayHour = df.groupby(['Day_of_Week','Hour']).count()["Reason"].unstack()
dayHour

** Now create a HeatMap using this new DataFrame. **

In [ ]:
plt.figure(figsize=(15,6))
sns.heatmap(dayHour,cmap='coolwarm');
# sns.heatmap(dayHour,cmap='coolwarm', annot=True, fmt=".1f");

** Now create a clustermap using this DataFrame. **

In [ ]:
plt.figure(figsize=(15,6))
sns.clustermap(dayHour,cmap='viridis');

** Now repeat these same plots and operations, for a DataFrame that shows the Month as the column. **

In [ ]:
dayMonth = df.groupby(['Day_of_Week','Month']).Reason.count().unstack()
dayMonth

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(dayMonth,cmap="coolwarm");

In [ ]:
plt.figure(figsize=(10,6))
sns.clustermap(dayMonth,cmap="coolwarm");

**Continue exploring the Data however you see fit!**
# Great Job!